In [1]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from qiskit import *
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver, VQE
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, GroverOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal

In [2]:
###### Fucntion for creating random adjacency matrix. #########
def create_adjacency_matrix(number_of_vertices, probability):
    adj_matrix = np.ones((number_of_vertices,number_of_vertices))
    for i in range(0,number_of_vertices):
        for j in range(i,number_of_vertices):
            if random.random()>probability or i==j:
                adj_matrix[i,j]=0
                adj_matrix[j,i]=0
    return adj_matrix

In [3]:
def create_quadratic_program(n,adj_matrix):
    qubo = QuadraticProgram()
    linear_coefficients = list()
    summation_terms = dict()
    for i in range(n):
        qubo.binary_var('x'+ str(i) + 'R')
        linear_coefficients.append(-2)
        qubo.binary_var('x' + str(i) + 'G')
        linear_coefficients.append(-2)
        qubo.binary_var('x' + str(i) + 'B')
        linear_coefficients.append(-2)
    for i in range(n):
        for j in ['R','G','B']:
            summation_terms['x'+str(i)+j,'x'+str(i)+j] = summation_terms.get(('x'+str(i)+j,'x'+str(i)+j), 1)
        summation_terms['x'+str(i)+'R','x'+str(i)+'G'] = summation_terms.get(('x'+str(i)+'R','x'+str(i)+'G'), 1)
        summation_terms['x'+str(i)+'G','x'+str(i)+'B'] = summation_terms.get(('x'+str(i)+'G','x'+str(i)+'B'), 1)
        summation_terms['x'+str(i)+'R','x'+str(i)+'B'] = summation_terms.get(('x'+str(i)+'R','x'+str(i)+'B'), 1)
    for i in range(n-1):
        for j in range(i+1,n):
            if adj_matrix[i,j]==1:
                summation_terms['x'+str(i)+'R','x'+str(j)+'R'] = summation_terms.get(('x'+str(i)+'R','x'+str(j)+'R'), 1)
                summation_terms['x'+str(i)+'G','x'+str(j)+'G'] = summation_terms.get(('x'+str(i)+'G','x'+str(j)+'G'), 1)
                summation_terms['x'+str(i)+'B','x'+str(j)+'B'] = summation_terms.get(('x'+str(i)+'B','x'+str(j)+'B'), 1)
    qubo.minimize(constant = n, linear= linear_coefficients, quadratic= summation_terms)
    #print(qubo.export_as_lp_string())
    return qubo
matrix = np.array([[0,1,1,1,1],[1,0,1,0,0],[1,1,0,1,0],[1,0,1,0,1],[1,0,0,1,0]])
create_quadratic_program(5,create_adjacency_matrix(5,0.5))

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 2 x0R - 2 x0G - 2 x0B - 2 x1R - 2 x1G - 2 x1B - 2 x2R - 2 x2G - 2 x2B
      - 2 x3R - 2 x3G - 2 x3B - 2 x4R - 2 x4G - 2 x4B + [ 2 x0R^2 + 2 x0R*x0G
      + 2 x0R*x0B + 2 x0R*x1R + 2 x0G^2 + 2 x0G*x0B + 2 x0G*x1G + 2 x0B^2
      + 2 x0B*x1B + 2 x1R^2 + 2 x1R*x1G + 2 x1R*x1B + 2 x1R*x2R + 2 x1R*x3R
      + 2 x1R*x4R + 2 x1G^2 + 2 x1G*x1B + 2 x1G*x2G + 2 x1G*x3G + 2 x1G*x4G
      + 2 x1B^2 + 2 x1B*x2B + 2 x1B*x3B + 2 x1B*x4B + 2 x2R^2 + 2 x2R*x2G
      + 2 x2R*x2B + 2 x2G^2 + 2 x2G*x2B + 2 x2B^2 + 2 x3R^2 + 2 x3R*x3G
      + 2 x3R*x3B + 2 x3R*x4R + 2 x3G^2 + 2 x3G*x3B + 2 x3G*x4G + 2 x3B^2
      + 2 x3B*x4B + 2 x4R^2 + 2 x4R*x4G + 2 x4R*x4B + 2 x4G^2 + 2 x4G*x4B
      + 2 x4B^2 ]/2 + 5
Subject To

Bounds
 0 <= x0R <= 1
 0 <= x0G <= 1
 0 <= x0B <= 1
 0 <= x1R <= 1
 0 <= x1G <= 1
 0 <= x1B <= 1
 0 <= x2R <= 1
 0 <= x2G <= 1
 0 <= x2B <= 1
 0 <= x3R <= 1
 0 <= x3G <= 1
 0 <= x3B <= 1
 0 <= 

In [4]:
def result_exact(qubo):
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(qubo)
    return exact_result.fval
qubo = create_quadratic_program(5,create_adjacency_matrix(5,0.5))
result_exact(qubo)


0.0

In [5]:
######### Function for getting cost and time vs n using NumpyeigenSolver ##########

def get_cost_and_time_numpy(n1, n2):
    cost_vs_n_fromnumpy = list()
    time_vs_n_fromnumpy = list()
    cost = list()
    tim = list()
    for j in range(n1,n2):
        for i in range(5):
            initial_time = time.time()
            qubo = Create_quadratic_program(16, create_edge_matrix(16,0.5))
            cost.append(result_exact(qubo))
            final_time = time.time()
            tim.append(final_time - initial_time)
        average_cost = sum(cost)/5
        average_time = sum(tim)/5
        (x, y) = (j, average_cost)
        (x, z) = (j, average_time)
        cost_vs_n_fromnumpy.append((x, y))
        time_vs_n_fromnumpy.append((x, z))
        print("n=", x, "cost=", y, "time=", z)
        cost.clear()
        tim.clear()
    return cost_vs_n_fromnumpy, time_vs_n_fromnumpy

In [6]:
def result_qaoa(qubo):
    qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('qasm_simulator'))
    qaoa = MinimumEigenOptimizer(qaoa_mes)
    qaoa_result = qaoa.solve(qubo)
    return qaoa_result.fval
qubo = create_quadratic_program(5,create_adjacency_matrix(5,0.5))
#result_qaoa(qubo)

In [7]:
######### Function for getting cost and time vs n using QAOA ##########

def get_cost_and_time_QAOA(n1, n2):
    cost_vs_n_fromQAOA = list()
    time_vs_n_fromQAOA = list()
    cost = list()
    tim = list()
    for j in range(n1,n2):
        for i in range(5):
            initial_time = time.time()
            op, offset, qubo = Create_quadratic_program(5, create_edge_matrix(5,0.5))
            cost.append(result_qaoa(qubo))
            final_time = time.time()
            tim.append(final_time - initial_time)
        average_cost = sum(cost)/5
        average_time = sum(tim)/5
        (x, y) = (j, average_cost)
        (x, z) = (j, average_time)
        cost_vs_n_fromQAOA.append((x, y))
        time_vs_n_fromQAOA.append((x, z))
        print("n=", x, "cost=", y, "time=", z)
        cost.clear()
        tim.clear()
    return cost_vs_n_fromQAOA, time_vs_n_fromQAOA

In [8]:
def using_vqe(qubo):
    vqe = VQE(quantum_instance=BasicAer.get_backend('qasm_simulator'))
    vqe_optimizer = MinimumEigenOptimizer(vqe)
    result = vqe_optimizer.solve(qubo)
    return result.fval
qubo = create_quadratic_program(5,create_adjacency_matrix(5,0.5))
#using_vqe(qubo)

In [9]:
######### Function for getting cost and time vs n using VQE ##########

def get_cost_and_time_VQE(n1, n2):
    cost_vs_n_fromVQE = list()
    time_vs_n_fromVQE = list()
    cost = list()
    tim = list()
    for j in range(n1,n2):
        for i in range(5):
            initial_time = time.time()
            op, offset, qubo = Create_quadratic_program(5, create_edge_matrix(5,0.5))
            cost.append(using_vqe(qubo))
            final_time = time.time()
            tim.append(final_time - initial_time)
        average_cost = sum(cost)/5
        average_time = sum(tim)/5
        (x, y) = (j, average_cost)
        (x, z) = (j, average_time)
        cost_vs_n_fromVQE.append((x, y))
        time_vs_n_fromVQE.append((x, z))
        print("n=", x, "cost=", y, "time=", z)
        cost.clear()
        tim.clear()
    return cost_vs_n_fromVQE, time_vs_n_fromVQE

In [10]:
def dec_to_bin(decimal, no_bits):
    binary = np.zeros((no_bits))
    for i in range(no_bits-1,-1,-1):
        #print(decimal, i)
        if int(decimal/(2**i)):
            binary[i]=1
        decimal = decimal%(2**i)
    return binary[::-1]

In [11]:
######## solving using Brute force ###########

def brute_force(n, adj_matrix):
    best_cost = 0
    best_case =None
    flag = True
    for i in range(2**n):
        binary = dec_to_bin(i,n)
        ans = sum(binary)
        for j in range(n-1):
            for k in range(j+1,n):
                if edge_matrix[j,k]==1:
                    ans = ans - 4*n*binary[j]*binary[k]
        if ans>best_cost:
            best_cost = ans
            #best_case = binary
    return best_cost